In [61]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial as P

In [62]:
def convoperation(image, kernels):
    result= np.copy(image)
    for kernel in kernels:
        result = cv2.filter2D(result, -1, kernel)
    return result

In [63]:
def color_segmentation(image):
    # rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    lower_yellow = np.array([18, 95, 95], dtype=np.uint8)
    upper_yellow = np.array([105, 255, 255], dtype=np.uint8)

    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([255, 80, 255], dtype=np.uint8)

    mask_yellow = cv2.inRange(image, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(image, lower_white, upper_white)

    combined_mask = cv2.bitwise_or(mask_yellow, mask_white)

    # segmented_image = cv2.bitwise_and(image, image, mask=combined_mask)
    # combined_mask = cv2.bitwise_or(mask_yellow, mask_white)
    inverted_mask = cv2.bitwise_not(combined_mask)
    
    masked_color_image = np.full_like(image, [0,255,0] , dtype=np.uint8)

    # Combine the masked color image with the original image using the inverted mask
    segmented_image = cv2.bitwise_or(cv2.bitwise_and(image, image, mask=inverted_mask), 
                                      cv2.bitwise_and(masked_color_image, masked_color_image, mask=combined_mask))

    # img = cv2.bitwise_and(image, image, mask=cv2.bitwise_or(mask_white, mask_yellow))
    return segmented_image

In [64]:
def warp(img):
    img_size = (img.shape[1], img.shape[0])

    pt1 = np.float32([[685, 450],
          [1090, 710],
          [220, 710],
          [595, 450]])

    pt2 = np.float32(
        [[900, 0],
          [900, 710],
          [250, 710],
          [250, 0]])

    M = cv2.getPerspectiveTransform(pt1, pt2)
    Minv = cv2.getPerspectiveTransform(pt2, pt1)

    binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    return binary_warped, Minv

In [65]:
def unwarp(warped_img, Minv):
    img_size = (warped_img.shape[1], warped_img.shape[0])
    unwarped_img = cv2.warpPerspective(warped_img, Minv, img_size, flags=cv2.INTER_LINEAR)
    return unwarped_img

In [66]:
def slope(line):
    return (float(line[3]) - line[1]) / (float(line[2]) - line[0])

In [67]:
PREV_LEFT_X1 = None
PREV_LEFT_X2 = None
PREV_RIGHT_X1 = None
PREV_RIGHT_X2 = None

BASE_IMG = None
CANNY_IMG = None

In [68]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=7):
    global PREV_LEFT_X1, PREV_LEFT_X2, PREV_RIGHT_X1, PREV_RIGHT_X2
    left_x = []
    left_y = []
    right_x = []
    right_y = []

    for line in lines:
        line = line[0]
        s = slope(line)

        if 0.3 > s > -0.3:
            continue

        if s < 0:
            if line[0] > img.shape[1] / 2 + 40:
                continue

            left_x += [line[0], line[2]]
            left_y += [line[1], line[3]]
            # cv2.line(img, (int(line[0]), int(line[1])), (int(line[2]), int(line[3])), [0, 0, 255], thickness)
        else:
            if line[0] < img.shape[1] / 2 - 40:
                continue

            right_x += [line[0], line[2]]
            right_y += [line[1], line[3]]
            # cv2.line(img, (int(line[0]), int(line[1])), (int(line[2]), int(line[3])), [255, 255, 0], thickness)

    y1 = img.shape[0]
    y2 = img.shape[0] / 2 + 90
    if len(left_x) <= 1 or len(right_x) <= 1:
        if PREV_LEFT_X1 is not None:
            cv2.line(img, (int(PREV_LEFT_X1), int(y1)), (int(PREV_LEFT_X2), int(y2)), color, thickness)
            cv2.line(img, (int(PREV_LEFT_X2), int(y1)), (int(PREV_RIGHT_X2), int(y2)), color, thickness)
        return

    left_poly = P.fit(np.array(left_x), np.array(left_y), 1)
    right_poly = P.fit(np.array(right_x), np.array(right_y), 1)

    left_x1 = (left_poly - y1).roots()
    right_x1 = (right_poly - y1).roots()

    left_x2 = (left_poly - y2).roots()
    right_x2 = (right_poly - y2).roots()

    if PREV_LEFT_X1 is not None:
        left_x1 = PREV_LEFT_X1 * 0.7 + left_x1 * 0.3
        left_x2 = PREV_LEFT_X2 * 0.7 + left_x2 * 0.3
        right_x1 = PREV_RIGHT_X1 * 0.7 + right_x1 * 0.3
        right_x2 = PREV_RIGHT_X2 * 0.7 + right_x2 * 0.3

    PREV_LEFT_X1 = left_x1
    PREV_LEFT_X2 = left_x2
    PREV_RIGHT_X1 = right_x1
    PREV_RIGHT_X2 = right_x2

    cv2.line(img, (int(left_x1), int(y1)), (int(left_x2), int(y2)), color, thickness)
    cv2.line(img, (int(right_x1), int(y1)), (int(right_x2), int(y2)), color, thickness)

In [69]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=1., β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [70]:
video = cv2.VideoCapture("videos/project_video.mp4")
# video = cv2.VideoCapture("videos/harder_challenge_video.mp4")
# video = cv2.VideoCapture("videos/challenge_video.mp4")
while True:
    ret, frame = video.read()
    if not ret:
        break
    base_img=frame
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame=cv2.GaussianBlur(frame, (3, 3), 0)
    # frame=cv2.Canny(frame,30, 130)
    wrapped,wrap2= warp(frame)
    f = color_segmentation(wrapped)
    f=cv2.cvtColor(f,cv2.COLOR_HSV2RGB)
    im=unwarp(f,wrap2)
    im=cv2.Canny(im,30,130)
    image = hough_lines(im, 1, np.pi / 90, 10, 15, 10)
    image=weighted_img(image, base_img, β=250.)
    cv2.imshow("final",image)
    cv2.imshow('Video', frame)
    # cv2.imshow('kernel',kerenel_edge)
    cv2.imshow('wrapped',wrapped)
    cv2.imshow('color',f)
    cv2.imshow('unwarped',im)
    # cv2.imshow('wrapped2',wrap2)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_35400\2039104347.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return (float(line[3]) - line[1]) / (float(line[2]) - line[0])
